In [1]:
import os
import json
import shutil
import logging
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import backend as K
from tqdm import tqdm


W0616 12:08:15.217847 139739548358464 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
%matplotlib inline


logger = logging.getLogger()
logger.setLevel(logging.INFO)

tf.logging.set_verbosity(tf.logging.INFO) 
tf.enable_resource_variables()

PARAMETERS

The values used in the base model of transformer were; num_layers=6, d_model = 512, dff = 2048. See the paper for all the other versions of the transformer.


In [3]:
from config import config


BERT_MODEL_URL = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

logging.info('Job Configuration:\n' + str(config))

I0616 12:08:16.332240 139739548358464 <ipython-input-3-cc026df3b2cc>:6] Job Configuration:
BATCH_SIZE: 2
BUFFER_SIZE: 1
CHECKPOINTDIR: checkpoint
DROPOUT_RATE: 0.1
D_FF: 2048
D_MODEL: 768
INITIAL_LR: 0.003
INPUT_SEQ_LEN: 512
LOGDIR: log
MAX_EXAMPLE_LEN: null
NUM_EPOCHS: 3
NUM_HEADS: 8
NUM_LAYERS: 8
OUTPUT_SEQ_LEN: 75
VOCAB_SIZE: 30522
WARMUP_STEPS: 4000



In [4]:
from models.abstractive_summarizer import AbstractiveSummarization
from models.abstractive_summarizer import train
from models.abstractive_summarizer import eval


    
model = AbstractiveSummarization(
    num_layers=config.NUM_LAYERS,
    d_model=config.D_MODEL,
    num_heads=config.NUM_HEADS,
    dff=config.D_FF,
    vocab_size=config.VOCAB_SIZE,
    input_seq_len=config.INPUT_SEQ_LEN,
    output_seq_len=config.OUTPUT_SEQ_LEN,
    rate=config.DROPOUT_RATE
)

I0616 12:08:16.348723 139739548358464 resolver.py:79] Using /tmp/tfhub_modules to cache modules.


Instructions for updating:
Colocations handled automatically by placer.


W0616 12:08:17.249488 139739548358464 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0616 12:08:19.532262 139739548358464 saver.py:1489] Saver not created because there are no variables in the graph to restore
I0616 12:08:20.572359 139739548358464 abstractive_summarizer.py:52] Extracting pretrained word embeddings weights from BERT
I0616 12:08:26.385534 139739548358464 abstractive_summarizer.py:63] Embedding matrix shape '(30522, 768)'


In [5]:
from data.load import load_cnn_dailymail


train_dataset, val_dataset, test_dataset, n_train_examples, n_val_examples, n_test_examples = load_cnn_dailymail()

n_train_batches = n_train_examples // config.BATCH_SIZE
n_val_batches = n_val_examples // config.BATCH_SIZE
n_test_batches = n_test_examples // config.BATCH_SIZE

logging.info(f"'{n_train_examples}' training examples, '{n_train_batches}' batches")
logging.info(f"'{n_val_examples}' validation examples, '{n_val_batches}' batches")
logging.info(f"'{n_test_examples}' testing examples, '{n_test_batches}' batches")

I0616 12:08:26.406254 139739548358464 dataset_builder.py:526] No config specified, defaulting to first: cnn_dailymail/plain_text
I0616 12:08:26.407960 139739548358464 dataset_builder.py:174] Overwrite dataset info from restored data version.
I0616 12:08:26.411083 139739548358464 dataset_builder.py:216] Reusing dataset cnn_dailymail (/home/ec2-user/tensorflow_datasets/cnn_dailymail/plain_text/0.0.2)
I0616 12:08:26.536819 139739548358464 <ipython-input-5-4d53a75058dd>:10] '287113' training examples, '143556' batches
I0616 12:08:26.537620 139739548358464 <ipython-input-5-4d53a75058dd>:11] '13368' validation examples, '6684' batches
I0616 12:08:26.538442 139739548358464 <ipython-input-5-4d53a75058dd>:12] '11490' testing examples, '5745' batches


In [6]:
import os
import json
import math
import shutil
import logging
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import backend as K
from tensorflow.python.keras.initializers import Constant

from tqdm import tqdm
from config import config

from data.load import load_cnn_dailymail

from random import randint
from rouge import Rouge

from ops.tokenization import tokenizer
from ops.tokenization import convert_idx_to_token_tensor

from ops.session import initialize_vars
from ops.session import save_variable_specs

from ops.metrics import calculate_rouge
from ops.tensor import with_column
from ops.regularization import label_smoothing
from ops.optimization import noam_scheme

from models.abstractive_summarizer import AbstractiveSummarization

In [ ]:
tvs = tf.trainable_variables()

for i in tvs:
    print(i)

In [8]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()
x0, x1, x2, y0, y1, y2 = el

xs = (x0, x1, x2)
ys = (y0, y1, y2)

train_loss, train_op, global_step, train_summaries = train(model, xs, ys)

with tf.Session() as sess:
    sess.run(iter.initializer)  
            


I0616 12:08:26.575592 139739548358464 abstractive_summarizer.py:437] Building Training Graph


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0616 12:08:29.729345 139739548358464 saver.py:1489] Saver not created because there are no variables in the graph to restore
I0616 12:08:29.867604 139739548358464 abstractive_summarizer.py:99] Building:'Draft summary'


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0616 12:08:35.134247 139739548358464 deprecation.py:506] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/keras/layers/core.py:143: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


W0616 12:08:38.071396 139739548358464 deprecation.py:323] From /home/ec2-user/SageMaker/bert-summarization/models/abstractive_summarizer.py:114: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
I0616 12:08:38.081864 139739548358464 abstractive_summarizer.py:226] Building: 'Refined Summary'


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0616 12:08:38.787830 139739548358464 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [7]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)  
#     tmp = sess.run(el)
    x0, x1, x2, y0, y1, y2 = el
    
    for i in [x0, x1, x2, y0, y1, y2]:
        print(sess.run(i).shape)
            
#     a, b= model((x0,x1,x2))
    
    logits_draft_summary, preds_draft_summary, draft_attention_dist, logits_refined_summary, preds_refined_summary, refined_attention_dist = model((x0,x1,x2))
        
    initialize_vars(sess)
    
    a= sess.run([preds_refined_summary])
    print(a.shape)
    print(a)
#     print()
#     print(b.shape)
#     print(b) 
#     print()  
#     print(c.shape)
#     print(c)     
#     print(c.shape, '\n')
#     print(c)     
#     print(d.shape, '\n')
#     print(d)         

(2, 512)
(2, 512)
(2, 512)
(2, 76)
(2, 76)
(2, 76)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0615 19:26:48.006563 139794197460800 saver.py:1489] Saver not created because there are no variables in the graph to restore
I0615 19:26:48.143085 139794197460800 abstractive_summarizer.py:123] Building: 'Greedy Draft Summary'
  0%|          | 0/74 [00:00<?, ?it/s]

Instructions for updating:
Use tf.cast instead.


W0615 19:26:56.669544 139794197460800 deprecation.py:323] From /home/ec2-user/SageMaker/bert-summarization/models/abstractive_summarizer.py:157: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
100%|██████████| 74/74 [03:06<00:00,  2.30s/it]
I0615 19:29:54.509504 139794197460800 abstractive_summarizer.py:302] Building: 'Greedy Refined Summary'
  0%|          | 0/74 [00:00<?, ?it/s]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0615 19:37:46.365857 139794197460800 saver.py:1489] Saver not created because there are no variables in the graph to restore
 99%|█████████▊| 73/74 [07:55<00:07,  7.84s/it]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0615 19:37:54.291556 139794197460800 saver.py:1489] Saver not created because there are no variables in the graph to restore
100%|██████████| 74/74 [08:03<00:00,  7.87s/it]


AttributeError: 'list' object has no attribute 'shape'

In [11]:
variables = tf.get_default_graph().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

for i in variables: print(i)

<tf.Variable 'abstractive_summarization/decoder/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'abstractive_summarization/decoder/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'abstractive_summarization/decoder/kernel_1:0' shape=(768, 768) dtype=float32>
<tf.Variable 'abstractive_summarization/decoder/bias_1:0' shape=(768,) dtype=float32>
<tf.Variable 'abstractive_summarization/decoder/kernel_2:0' shape=(768, 768) dtype=float32>
<tf.Variable 'abstractive_summarization/decoder/bias_2:0' shape=(768,) dtype=float32>
<tf.Variable 'abstractive_summarization/decoder/kernel_3:0' shape=(768, 768) dtype=float32>
<tf.Variable 'abstractive_summarization/decoder/bias_3:0' shape=(768,) dtype=float32>
<tf.Variable 'abstractive_summarization/decoder/kernel_4:0' shape=(768, 768) dtype=float32>
<tf.Variable 'abstractive_summarization/decoder/bias_4:0' shape=(768,) dtype=float32>
<tf.Variable 'abstractive_summarization/decoder/kernel_5:0' shape=(768, 768) dtype=float32>
<tf.Variable 'abstract

In [10]:
import math

from rouge import Rouge
from ops.session import initialize_vars
from ops.session import save_variable_specs


train_iterator = train_dataset.make_initializable_iterator()
train_stream = train_iterator.get_next()

# val_iterator = val_dataset.make_initializable_iterator()
# val_stream = val_iterator.get_next()

xs, ys = train_stream[:3], train_stream[3:]
train_loss, train_op, global_step, train_summaries = model.train(xs, ys)

# xs, ys = val_stream[:3], val_stream[3:]
# y, y_hat, eval_loss, eval_summaries = model.eval(xs, ys)

saver = tf.train.Saver(max_to_keep=config.NUM_EPOCHS)

I0613 19:06:54.430439 139843988014912 <ipython-input-9-ee212edc8049>:449] Building Training Graph


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0613 19:06:58.338555 139843988014912 saver.py:1489] Saver not created because there are no variables in the graph to restore
I0613 19:06:58.502389 139843988014912 <ipython-input-9-ee212edc8049>:112] Building:'Draft summary'


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0613 19:07:03.779280 139843988014912 deprecation.py:506] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/keras/layers/core.py:143: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


W0613 19:07:06.773543 139843988014912 deprecation.py:323] From <ipython-input-9-ee212edc8049>:127: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
I0613 19:07:06.784623 139843988014912 <ipython-input-9-ee212edc8049>:236] Building: 'Refined Summary'


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0613 19:07:07.623564 139843988014912 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert_layer (BertLayer)       multiple                  110104890 
_________________________________________________________________
embedding (Embedding)        multiple                  23440896  
_________________________________________________________________
decoder (Decoder)            multiple                  63023104  
_________________________________________________________________
dense (Dense)                multiple                  23471418  
Total params: 220,040,308
Trainable params: 86,494,522
Non-trainable params: 133,545,786
_________________________________________________________________


In [ ]:

# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:

    if os.path.isdir(config.LOGDIR):
        shutil.rmtree(config.LOGDIR)

    os.mkdir(config.LOGDIR)

    ckpt = tf.train.latest_checkpoint(config.CHECKPOINTDIR)

    rouge = Rouge()

    if ckpt is None:
        logging.info("Initializing from scratch")
        sess.run(tf.global_variables_initializer())
        save_variable_specs(os.path.join(config.LOGDIR, "specs"))
    else:
        saver.restore(sess, ckpt)        

    summary_writer_train = tf.summary.FileWriter(os.path.join(config.LOGDIR, 'train'), sess.graph)
    summary_writer_eval = tf.summary.FileWriter(os.path.join(config.LOGDIR, 'eval'), sess.graph)


    initialize_vars(sess)

    _gs = sess.run(global_step)

    sess.run(train_iterator.initializer)
#     sess.run(val_iterator.initializer)

    total_steps = config.NUM_EPOCHS * n_train_batches

    logger.info(f"Running Training Job for '{total_steps}' steps")
    
    for i in tqdm(range(_gs, total_steps+1)):

        _loss, _, _gs, _summary = sess.run([train_loss, train_op, global_step, train_summaries])

        epoch = math.ceil(_gs / n_train_batches)

        summary_writer_train.add_summary(_summary, _gs)
        summary_writer_train.flush() 

#         if (_gs and _gs % n_train_batches == 0):

#             logger.info(f"Epoch '{epoch}' done")
#             logger.info(f"Current training step: '{_gs}")

#             _y, _y_hat, _eval_summary = sess.run([y, y_hat, eval_summaries])

#             summary_writer_eval.add_summary(_eval_summary, 0)
#             summary_writer_eval.flush()       

#             # monitor a random sample
#             rnd = randint(0, _y.shape[0] - 1)

#             y_rnd = ' '.join(tokenizer.convert_ids_to_tokens(_y[rnd]))
#             y_hat_rnd = ' '.join(tokenizer.convert_ids_to_tokens(_y_hat[rnd]))

#             rouges = rouge.get_scores(y_rnd, y_hat_rnd)[0]
#             r1_val, r2_val, rl_val = rouges['rouge-1']["f"], rouges['rouge-2']["f"], rouges['rouge-l']["f"]

#             print('Target:')
#             print(y_rnd)
#             print('Prediction:')
#             print(y_hat_rnd)

#             print(f"ROUGE-1 '{r1_val}'")
#             print(f"ROUGE-2 '{r2_val}'")
#             print(f"ROUGE-L '{rl_val}'")
#             print(f"ROUGE-AVG '{np.mean([r1_val, r2_val, rl_val])}'", '\n--\n')

#             logging.info("Checkpoint: Saving Model")

#             model_output = f"abstractive_summarization_2019_epoch_{epoch}_loss_{str(round(_loss, 4))}"

#             ckpt_name = os.path.join(config.CHECKPOINTDIR, model_output)

#             saver.save(sess, ckpt_name, global_step=_gs)

#             logging.info(f"After training '{_gs}' steps, '{ckpt_name}' has been saved.")

    summary_writer_train.close()  
#     summary_writer_eval.close()  

I0613 19:10:56.591774 139843988014912 <ipython-input-14-76323c486f43>:16] Initializing from scratch
I0613 19:11:04.208605 139843988014912 session.py:44] Variables info has been saved.


num_params:  613275999


I0613 19:11:41.390239 139843988014912 <ipython-input-14-76323c486f43>:35] Running Training Job for '114844' steps
  0%|          | 38/114845 [05:22<236:57:04,  7.43s/it]

In [ ]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)  
#     tmp = sess.run(el)
    x0, x1, x2, y0, y1, y2 = el
    
    a, b = model((x0,x1,x2), (y0, y1, y2), True)
    
    initialize_vars(sess)
    
    a,b = sess.run([a,b])
    print(a.shape)
    print(a)
    print()
    print(b.shape)
    print(b) 
    print()
#     print(c.shape)
#     print(c)     
#     print(c.shape, '\n')
#     print(c)     
#     print(d.shape, '\n')
#     print(d)         

In [ ]:
with open(b'/tmp/tfhub_modules/5a395eafef2a37bd9fc55d7f6ae676d2a134a838/assets/vocab.txt') as f:
    data = f.readlines()


In [15]:


model.summary()

I0608 01:01:57.134044 139913473955648 <ipython-input-14-9abbb652388f>:377] Building Evaluation Graph


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0608 01:02:01.785801 139913473955648 saver.py:1489] Saver not created because there are no variables in the graph to restore
I0608 01:02:01.973906 139913473955648 <ipython-input-14-9abbb652388f>:98] Building: 'Greedy Draft Summary'
100%|██████████| 2/2 [00:10<00:00,  6.23s/it]
I0608 01:02:12.164133 139913473955648 <ipython-input-14-9abbb652388f>:196] Building: 'Greedy Refined Summary'
  0%|          | 0/2 [00:00<?, ?it/s]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0608 01:02:13.135066 139913473955648 saver.py:1489] Saver not created because there are no variables in the graph to restore
 50%|█████     | 1/2 [00:03<00:03,  3.30s/it]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0608 01:02:16.977452 139913473955648 saver.py:1489] Saver not created because there are no variables in the graph to restore
100%|██████████| 2/2 [00:07<00:00,  3.48s/it]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert_layer_1 (BertLayer)     multiple                  110104890 
_________________________________________________________________
embedding_1 (Embedding)      multiple                  23440896  
_________________________________________________________________
decoder_1 (Decoder)          multiple                  63023104  
_________________________________________________________________
dense_21 (Dense)             multiple                  23471418  
Total params: 220,040,308
Trainable params: 86,494,522
Non-trainable params: 133,545,786
_________________________________________________________________


In [16]:


with tf.Session() as sess:
    
    if os.path.isdir(config.LOGDIR):
        shutil.rmtree(config.LOGDIR)
    
    summary_writer = tf.summary.FileWriter(config.LOGDIR, sess.graph)
        
    initialize_vars(sess)
    
    for i in range(1):
        
        sess.run(iterator.initializer)
        
        _y, _y_hat, _eval_summary = sess.run([y, y_hat, eval_summaries])
        
        summary_writer.add_summary(_eval_summary, 0)
        summary_writer.flush()       
        
        # monitor a random sample
        rnd = randint(0, _y.shape[0] - 1)

        print('Target:')
        print(' '.join(tokenizer.convert_ids_to_tokens(_y[rnd])))
        print('Prediction:')
        print(' '.join(tokenizer.convert_ids_to_tokens(_y_hat[rnd])))
        
    summary_writer.close()        

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[node IteratorGetNext (defined at <ipython-input-11-04a087f227a6>:2) ]]

Caused by op 'IteratorGetNext', defined at:
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/kernelapp.py", lin

In [ ]:
def loss_function(pred, real):
    """
    Since the target sequences are padded,
    it is important to apply a padding mask when calculating the loss.
    """
    logits_draft_summary, logits_refined_summary = pred
    
    target_ids_ = label_smoothing(tf.one_hot(real, depth=self.vocab_size))

    # use right shifted target
    loss_draft = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_draft_summary, labels=target_ids_[:, 1:])

    # use non-shifted target (we want to predict the masked word)
    loss_refined = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_refined_summary, labels=target_ids_[:, :-1])

    loss = loss_draft + loss_refined    
    
    mask = tf.math.logical_not(tf.math.equal(real, 0))    
    mask = tf.cast(mask, dtype=loss_.dtype)
    
    loss *= mask

    return tf.reduce_mean(loss)  


In [34]:
batch_size = 5
vocab_size = 30522
beam_size = 3
alpha = 0.8
max_seqlen = 10
EOS_ID = 102

initial_ids = tf.zeros([batch_size], dtype=tf.int32)

decoded_ids, scores = sequence_beam_search(
    symbols_to_logits_fn=None,
    initial_ids=initial_ids,
    initial_cache=None,
    vocab_size=vocab_size,
    beam_size=beam_size,
    alpha=alpha,
    max_decode_length=max_seqlen,
    eos_id=EOS_ID)

# with tf.Session() as sess:
#     print(tf.trainable_variables())

ImportError: 

In [ ]:
def mask_each_timestamp(x, mask_with):
    """
    !!! CURRENTLY NOT USED
    
    Masks each word in the summary draft one by one with the [MASK] token
    At t-th time step the t-th word of input summary is
    masked, and the decoder predicts the refined word given other
    words of the summary.
    
    x :: (N, T)
    returrn :: (N, T-2, T)
    
    We do not mask the first and last postition (corresponding to [CLS] and [SEP])
    """

    N, T = tf.shape(x)[0], tf.shape(x)[1]

    first = tf.reshape(tf.tile(x[:, 0], [T-2]), [N, T-2, 1])
    last = tf.reshape(tf.tile(x[:, -1], [T-2]), [N, T-2, 1])
    
    x = x[:, 1:-1]
    T = T - 2
    
    masked = tf.reshape(tf.tile(x, [1, T]), [N, T, T])
    
    diag = tf.ones([N, T], dtype=masked.dtype) * mask_with
    masked = tf.linalg.set_diag(masked, diag)
    
    masked = tf.concat([first, masked, last], axis=2)
    
    return masked

    
with tf.Session() as sess:
    
    tensor = tf.constant([[1,2,3], [4,5,6], [4,5,6], [4,5,6], [4,5,6]])
    
    print(sess.run( tf.random_uniform((), 0, tf.shape(tensor)[0], tf.int32)))
    

In [13]:

_gs

global_step = tf.train.get_or_create_global_step()

learning_rate = noam_scheme(initial_lr, global_step, warmup_steps)

optimizer = tf.train.AdamOptimizer(learning_rate, beta1=0.9, beta2=0.98, epsilon=1e-9)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# train_op = optimizer.minimize(loss, global_step=global_step)

model.compile(
    optimizer=optimizer,
    loss=loss_function,
    metrics=[tf.keras.metrics.categorical_accuracy]
)

TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.

In [42]:
isinstance(model, tf.keras.Model)

True

In [ ]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)  
#     tmp = sess.run(el)
    x0, x1, x2, y0, y1, y2 = el
    
    a,b,_,c,d,_ = model((x0,x1,x2))
    
    initialize_vars(sess)
    
    a,b,c,d = sess.run([a,b,c,d])
    print(a.shape, '\n')
    print(a)
    print(b.shape, '\n')
    print(b) 
    print(c.shape, '\n')
    print(c)     
    print(d.shape, '\n')
    print(d)         

In [39]:
with tf.Session() as sess:
    print(tf.trainable_variables())


[<tf.Variable 'Adam/iterations:0' shape=() dtype=int64>, <tf.Variable 'Adam/lr:0' shape=() dtype=float32>, <tf.Variable 'Adam/beta_1:0' shape=() dtype=float32>, <tf.Variable 'Adam/beta_2:0' shape=() dtype=float32>, <tf.Variable 'Adam/decay:0' shape=() dtype=float32>, <tf.Variable 'Adam_1/iterations:0' shape=() dtype=int64>, <tf.Variable 'Adam_1/lr:0' shape=() dtype=float32>, <tf.Variable 'Adam_1/beta_1:0' shape=() dtype=float32>, <tf.Variable 'Adam_1/beta_2:0' shape=() dtype=float32>, <tf.Variable 'Adam_1/decay:0' shape=() dtype=float32>, <tf.Variable 'Adam_2/iterations:0' shape=() dtype=int64>, <tf.Variable 'Adam_2/lr:0' shape=() dtype=float32>, <tf.Variable 'Adam_2/beta_1:0' shape=() dtype=float32>, <tf.Variable 'Adam_2/beta_2:0' shape=() dtype=float32>, <tf.Variable 'Adam_2/decay:0' shape=() dtype=float32>]


In [30]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()
model.fit(el, epochs=10, steps_per_epoch=30)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 6 arrays: [<tf.Tensor 'IteratorGetNext_1:0' shape=(?, ?) dtype=int64>, <tf.Tensor 'IteratorGetNext_1:1' shape=(?, ?) dtype=int64>, <tf.Tensor 'IteratorGetNext_1:2' shape=(?, ?) dtype=int64>, <tf.Tensor 'Iterato...

In [14]:
with tf.Session() as sess:
    initialize_vars(sess)    
    model.fit(train_dataset, epochs=1, steps_per_epoch=1)

TypeError: not all arguments converted during string formatting

In [10]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)  
#     tmp = sess.run(el)
    x0, x1, x2, y0, y1, y2 = el
    print(x0.shape, x1.shape, x2.shape)
    print(y0.shape, y1.shape, y2.shape)
    
    a = model((x0,x1,x2), (y0,y1,y2), True)
    
    initialize_vars(sess)
#     for i in m.trainable_weights: print(i)
#     raise ValueError
    
    a = sess.run([a])
    print(a)    



(?, ?) (?, ?) (?, ?)
(?, ?) (?, ?) (?, ?)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0602 02:27:36.021267 140483075295040 saver.py:1489] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0602 02:27:37.085570 140483075295040 deprecation.py:506] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/keras/layers/core.py:143: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0602 02:27:49.526523 140483075295040 saver.py:1489] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0602 02:27:53.526570 140483075295040 saver.py:1489] Saver not created because there are no variables in the graph to restore


[20.578041]


In [ ]:
# tf.summary.scalar('lr', lr)
# tf.summary.scalar("loss", loss)
# tf.summary.scalar("global_step", global_step)

# summaries = tf.summary.merge_all()

warmup_steps = 4000
initial_lr = 0.0003

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = model(inputs, targets)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)


def noam_scheme(init_lr, global_step, warmup_steps=4000.):
    '''Noam scheme learning rate decay
    init_lr: initial learning rate. scalar.
    global_step: scalar.
    warmup_steps: scalar. During warmup_steps, learning rate increases
        until it reaches init_lr.
    '''
    step = tf.cast(global_step + 1, dtype=tf.float32)
    return init_lr * warmup_steps ** 0.5 * tf.minimum(step * warmup_steps ** -1.5, step ** -0.5)


global_step = tf.train.get_or_create_global_step()

learning_rate = noam_scheme(initial_lr, global_step, warmup_steps)

optimizer = tf.train.AdamOptimizer(learning_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)


iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)  
#     tmp = sess.run(el)
    x0, x1, x2, y0, y1, y2 = el
    print(x0.shape, x1.shape, x2.shape)
    print(y0.shape, y1.shape, y2.shape)
    
    loss_value, grads = grad(model, (x0,x1,x2), (y0,y1,y2))
    
    print(loss_value)
    print(grads)

    initialize_vars(sess)



In [14]:
in_id = tf.keras.layers.Input(shape=(MAX_EXAMPLE_LEN,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(MAX_EXAMPLE_LEN,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(MAX_EXAMPLE_LEN,), name="segment_ids")

warmup_steps = 4000
initial_lr = 0.0003

def noam_scheme(init_lr, global_step, warmup_steps=4000.):
    '''Noam scheme learning rate decay
    init_lr: initial learning rate. scalar.
    global_step: scalar.
    warmup_steps: scalar. During warmup_steps, learning rate increases
        until it reaches init_lr.
    '''
    step = tf.cast(global_step + 1, dtype=tf.float32)
    return init_lr * warmup_steps ** 0.5 * tf.minimum(step * warmup_steps ** -1.5, step ** -0.5)


global_step = tf.train.get_or_create_global_step()

learning_rate = noam_scheme(initial_lr, global_step, warmup_steps)

optimizer = tf.train.AdamOptimizer(learning_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.compile(optimizer=optimizer)

In [4]:
tmp = tf.constant([[[1,2,3,4], [5,6,7,8]]])

def loss_function(pred, real):
    """
    Since the target sequences are padded,
    it is important to apply a padding mask when calculating the loss.
    """
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=real)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)    



with tf.Session() as sess:
    print(sess.run())


Instructions for updating:
Colocations handled automatically by placer.


W0601 17:55:21.698049 140022164780864 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


FailedPreconditionError: Error while reading resource variable global_step from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/global_step)
	 [[node global_step/Read/ReadVariableOp (defined at <ipython-input-4-76a7a9e69621>:3) ]]

Caused by op 'global_step/Read/ReadVariableOp', defined at:
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ec2-

The negative log likelihood loss. It is useful to train a classification problem with `C` classes.

Computes sparse softmax cross entropy between `logits` and `labels`. Measures the probability error in discrete classification tasks in which the
  classes are mutually exclusive (each entry is in exactly one class).

In [29]:
session = tf.Session()
print(session.run(tf.global_variables_initializer()))
print(session.run(tf.report_uninitialized_variables()))


None
[]


In [26]:
from ops.regularization import label_smoothing


ids = tf.constant([[0, 1, 0], [2, 1, 0]])

lgts = tf.constant([[[12.2, 20.2, 23.3], [2.2, 4.2, 12.1], [2.2, 4.2, 12.1]], [[10.1, 42.1, 24.1], [2.1, 5.1, 10.1], [10.1, 42.1, 24.1]]])

smooth_labels = label_smoothing(tf.one_hot(ids, depth=3))


# use right shifted target
lss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=lgts, labels=smooth_labels)


with tf.Session() as sess:
    print(sess.run(ids))
    print(sess.run(smooth_labels))
    print(sess.run(lss).shape)
    print(sess.run(tf.reduce_mean(lss)))
        
#     print(sess.run())


[[0 1 0]
 [2 1 0]]
[[[0.93333334 0.03333334 0.03333334]
  [0.03333334 0.93333334 0.03333334]
  [0.93333334 0.03333334 0.03333334]]

 [[0.03333334 0.03333334 0.93333334]
  [0.03333334 0.93333334 0.03333334]
  [0.93333334 0.03333334 0.03333334]]]
(2, 3)
13.498105


In [ ]:
from layers.bert import BertLayer


class BertLayer(tf.keras.layers.Layer):
    """
    Custom Keras layer, integrating BERT from tf-hub
    """
    def __init__(self, url, seq_len=512, d_embedding=768, n_fine_tune_layers=None, **kwargs):
        self.url = url
        self.n_fine_tune_layers = n_fine_tune_layers
        self.seq_len = seq_len
        self.d_embedding = d_embedding
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.url,
            trainable=self.trainable,
            name="{}_bert_module".format(self.name)
        )

        trainable_vars = self.bert.variables
        
        if self.trainable:

            # Remove unused layers
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]

            # Select how many layers to fine tune
            trainable_vars = trainable_vars[-self.n_fine_tune_layers :]

            # Add to trainable weights
            for var in trainable_vars:
                self._trainable_weights.append(var)

            for var in self.bert.variables:
                if var not in self._trainable_weights:
                    self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        
        input_ids, input_mask, segment_ids = inputs
        
        bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)["sequence_output"]
        return result

    def compute_output_shape(self, input_shape):
        raise ValueError()
        return (input_shape[0], self.seq_len, self.d_embedding)

    
    
in_id = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(SEQ_LEN,), name="segment_ids")

bert_inputs = [in_id, in_mask, in_segment]

bert = BertLayer(BERT_MODEL_URL, trainable=False)(bert_inputs)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=bert)

In [14]:
atmp1 = np.zeros((30522))
atmp1[10] = 1

atmp2 = np.zeros((30522))
atmp2[20] = 1

In [49]:
atmp = np.array([atmp1, atmp2], dtype=np.float32)
atmp.dtype

dtype('float32')

In [50]:
from tensorflow.python.keras.initializers import Constant


with tf.Session() as sess:
    
    initialize_vars(sess)
    embedding_matrix = sess.run(bert.variable_map['bert/embeddings/word_embeddings'])

target_vocab_size = len(data)
d_model = 768

embedding = tf.keras.layers.Embedding(target_vocab_size, d_model, trainable=False, embeddings_initializer=Constant(embedding_matrix))



In [59]:
embedding.compute_output_shape((10, target_vocab_size))

TensorShape([Dimension(10), Dimension(30522), Dimension(768)])

In [52]:
with tf.Session() as sess:
    
    initialize_vars(sess) 
    atmp_ = tf.convert_to_tensor(atmp, np.float32)
    print(atmp_)
    print(sess.run(embedding(atmp_)).shape)
    raise ValueError
    print(sess.run(tf.matmul(atmp, bert.variable_map['bert/embeddings/word_embeddings'])))
    print(sess.run(bert.variable_map['bert/embeddings/word_embeddings'](np.array([0, 1]))))


    

Tensor("Const_20:0", shape=(2, 30522), dtype=float32)


NotFoundError: Resource localhost/embedding_5/embeddings/N10tensorflow3VarE does not exist.
	 [[node embedding_5/embedding_lookup (defined at <ipython-input-52-fac9b5b8d76d>:6) ]]

Caused by op 'embedding_5/embedding_lookup', defined at:
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/ec2-user/anaconda3

In [53]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)
    initialize_vars(sess)    
    tmp = sess.run(el)
    x0, x1, x2, y0, y1, y2 = tmp
    print(x0.shape, x1.shape, x2.shape)
#     print(x0, '\n', x1)
    tmp2 = model.predict([x0, x1, x2])
    print(tmp2)
#     print(sess.run(tmp2))  # (batch_size, input_seq_len, d_model)    


    

(1, 512) (1, 512) (1, 512)


NameError: name 'model' is not defined

In [9]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:

#     sess.run(iter.initializer)
#     tmp = sess.run(el)
#     print(tmp)
    sample_transformer = Transformer(
        num_layers=2, d_model=512, num_heads=8, dff=2048, 
        input_vocab_size=8500, target_vocab_size=8000)

    temp_input = tf.random.uniform((64, 62))
    temp_target = tf.random.uniform((64, 26))

    fn_out, _ = sample_transformer(temp_input, temp_target, training=False, 
                                   enc_padding_mask=None, 
                                   look_ahead_mask=None,
                                   dec_padding_mask=None)
    initialize_vars(sess)
    
    print(sess.run(fn_out))  # (batch_size, input_seq_len, d_model)    
#     x0, x1, x2, y0, y1, y2 = tmp
#     tmp2 = model.predict([x0, x1, x2])
    

[[[ 0.49737298  0.5550515  -0.13341121 ... -0.5284789  -0.61509997
   -0.04155324]
  [ 0.5157517   0.5635947  -0.09799525 ... -0.5260042  -0.62772596
   -0.03477238]
  [ 0.53440964  0.5700865  -0.05099516 ... -0.5282975  -0.63185966
   -0.02259888]
  ...
  [ 0.39933348  0.4695823  -0.19357261 ... -0.49450147 -0.70106435
   -0.01530197]
  [ 0.41170758  0.47419053 -0.20090303 ... -0.5092494  -0.7242769
    0.0037366 ]
  [ 0.42569858  0.49202585 -0.1921086  ... -0.52926743 -0.73894566
    0.00663666]]

 [[ 0.49737298  0.5550515  -0.13341121 ... -0.5284789  -0.61509997
   -0.04155324]
  [ 0.5157517   0.5635947  -0.09799525 ... -0.5260042  -0.62772596
   -0.03477238]
  [ 0.53440964  0.5700865  -0.05099516 ... -0.5282975  -0.63185966
   -0.02259888]
  ...
  [ 0.39933348  0.4695823  -0.19357261 ... -0.49450147 -0.70106435
   -0.01530197]
  [ 0.41170758  0.47419053 -0.20090303 ... -0.5092494  -0.7242769
    0.0037366 ]
  [ 0.42569858  0.49202585 -0.1921086  ... -0.52926743 -0.73894566
    0.00

In [30]:
sess = tf.InteractiveSession()


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [6]:
def print_out(q, k, v):
    temp_out, temp_attn = scaled_dot_product_attention(q, k, v, None)
    print ('Attention weights are:')
    print (temp_attn.eval())
    print ('Output is:')
    print (temp_out.eval())

    
temp_k = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=tf.float32)  # (4, 2)

# This query aligns equally with the first and second key, 
# so their values get averaged.
temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32)  # (3, 3)
print_out(temp_q, temp_k, temp_v)




Attention weights are:


ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [7]:
temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
print(out.shape, attn.shape)

sample_ffn = point_wise_feed_forward_network(512, 2048)
sample_ffn(tf.random.uniform((64, 50, 512)))

NameError: name 'MultiHeadAttention' is not defined

models

In [29]:
sample_transformer = Transformer(
    num_layers=2, d_model=512, num_heads=8, dff=2048, 
    input_vocab_size=8500, target_vocab_size=8000)

temp_input = tf.random.uniform((64, 62))
temp_target = tf.random.uniform((64, 26))

fn_out, _ = sample_transformer(temp_input, temp_target, training=False, 
                               enc_padding_mask=None, 
                               look_ahead_mask=None,
                               dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)


TensorShape([Dimension(64), Dimension(26), Dimension(8000)])

### Optimizer

In [31]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    """
    Use the Adam optimizer with a custom learning rate scheduler according to the formula
    """
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


AttributeError: module 'tensorflow._api.v1.keras.optimizers' has no attribute 'schedules'

### Losses and Metrics

In [43]:
loss_object = tf.keras.losses.sparse_categorical_crossentropy(from_logits=True)


TypeError: sparse_categorical_crossentropy() missing 2 required positional arguments: 'y_true' and 'y_pred'

In [41]:
def loss_function(real, pred):
    """
    Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.
    """
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')


In [44]:
def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by 
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask


In [34]:
x0.shape

(1, 512)

In [33]:
x0[:, :-1].shape


(1, 511)

In [25]:
sample_decoder = Decoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, target_vocab_size=8000)

output, attn = sample_decoder(tf.random.uniform((64, 26)), 
                              enc_output=sample_encoder_output, 
                              training=False, look_ahead_mask=None, 
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape, attn['decoder_layer2_block1'].shape


(TensorShape([Dimension(64), Dimension(26), Dimension(512)]),
 TensorShape([Dimension(64), Dimension(8), Dimension(26), Dimension(62)]),
 TensorShape([Dimension(64), Dimension(8), Dimension(26), Dimension(26)]))

In [21]:
sample_encoder = Encoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, input_vocab_size=8500)

sample_encoder_output = sample_encoder(tf.random.uniform((64, 62)), 
                                       training=False, mask=None)

print (sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)


(64, 62, 512)


In [18]:
sample_decoder_layer = DecoderLayer(512, 8, 2048)

sample_decoder_layer_output, _, _ = sample_decoder_layer(
    tf.random.uniform((64, 50, 512)), sample_encoder_layer_output, 
    False, None, None)

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)


TensorShape([Dimension(64), Dimension(50), Dimension(512)])

In [14]:
in_id = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(SEQ_LEN,), name="segment_ids")

bert_inputs = [in_id, in_mask, in_segment]

bert = BertLayer(BERT_MODEL_URL, trainable=False)(bert_inputs)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=bert)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0523 12:59:35.651068 140526081922880 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [19]:
import tensorflow as tf


def create_padding_mask(seq):
    """
    Mask all the pad tokens in the batch of sequence.
    It ensures that the model does not treat padding as the input.
    The mask indicates where pad value 0 is present:
    it outputs a 1 at those locations, and a 0 otherwise.    
    """
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions so that we can add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


def create_look_ahead_mask(size):
    """
    The look-ahead mask is used to mask the future tokens in a sequence.
    In other words, the mask indicates which entries should not be used. 

    This means that to predict the third word, only the first and second word will be used.
    Similarly to predict the fourth word, only the first, second and the third word will be used and so on.
    """
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)


def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by 
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask



In [47]:
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])

with tf.Session() as sess:
    print(sess.run(create_padding_mask(x)).shape)


(3, 1, 1, 5)


In [48]:
create_masks(x0, y0)

(<tf.Tensor 'strided_slice_9:0' shape=(1, 1, 1, 512) dtype=float32>,
 <tf.Tensor 'Maximum_1:0' shape=(1, 1, 512, 512) dtype=float32>,
 <tf.Tensor 'strided_slice_10:0' shape=(1, 1, 1, 512) dtype=float32>)

In [42]:
x0[:, :-1]

array([[  101,  2028,  1997,  3725,  1005,  1055,  2087,  6450,  2406,
         5014,  2038,  2908,  2006,  1996,  3006,  1011,  1998,  2009,
         2428,  2003,  4906,  2005,  1037,  3035,  1012,  1996,  3694,
         1045,  3205,  2026, 13629,  2380,  1010,  2275,  1999,  1037,
         1015,  1010,  8380,  1011,  7456,  3776,  1010,  2038,  1037,
         2157,  2548,  3976,  6415,  1011,  1037, 22771, 14534,  2629,
         2213,  1012,  2021,  1996,  3776,  6074,  8304,  1996,  5096,
         2360,  2045,  2003,  2053, 15843,  1997,  4022, 17394,  5782,
         2000,  3193,  1996,  3200,  1012,  2188,  4906,  2005, 16664,
         1024,  2026, 13629,  2380,  2003,  3694,  1045,  3205,  1998,
         2001,  3294, 10601,  2058,  1037,  2093,  2095,  2622,  2011,
         1996,  2783,  5608,  1012,  1996,  3035,  2218,  1037, 12695,
         2182,  1999,  3541, 21136,  7430,  2005,  1017,  1010,  2199,
         6368,  1999,  2494,  1012,  2009,  2003,  2028,  1997,  3725,
      

In [15]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)

train_examples, val_examples = examples['train'], examples['validation']


I0528 14:56:33.385675 139957305521984 dataset_builder.py:177] Load pre-computed datasetinfo (eg: splits) from bucket.
I0528 14:56:33.650062 139957305521984 dataset_info.py:377] Loading info from GCS for ted_hrlr_translate/pt_to_en/0.0.1
I0528 14:56:34.125548 139957305521984 dataset_builder.py:236] Generating dataset ted_hrlr_translate (/home/ec2-user/tensorflow_datasets/ted_hrlr_translate/pt_to_en/0.0.1)


I0528 14:56:34.203127 139957305521984 download_manager.py:241] Downloading http://www.phontron.com/data/qi18naacl-dataset.tar.gz into /home/ec2-user/tensorflow_datasets/downloads/phontron.com_qi18naacl-datasetLVhDyvf9PQ-GjP4jim31ESZuvRjHI5wpNhR5SJvsNOo.tar.gz.tmp.85acbe6968a948c0ab60989f292e5947...
I0528 14:56:46.151685 139957305521984 dataset_builder.py:675] Generating split train
I0528 14:56:46.155997 139957305521984 file_format_adapter.py:271] Writing TFRecords


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


W0528 14:56:52.763825 139957305521984 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_datasets/core/file_format_adapter.py:247: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


I0528 14:56:53.236819 139957305521984 dataset_builder.py:675] Generating split validation
I0528 14:56:53.238343 139957305521984 file_format_adapter.py:271] Writing TFRecords


I0528 14:56:53.504857 139957305521984 dataset_builder.py:675] Generating split test
I0528 14:56:53.506487 139957305521984 file_format_adapter.py:271] Writing TFRecords


I0528 14:56:53.876027 139957305521984 dataset_builder.py:264] Skipping computing stats for mode ComputeStatsMode.AUTO.


Dataset ted_hrlr_translate downloaded and prepared to /home/ec2-user/tensorflow_datasets/ted_hrlr_translate/pt_to_en/0.0.1. Subsequent calls will reuse this data.


In [22]:
iter = train_examples.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)  
    x, y = sess.run(el)
    print(x.decode('utf-8'))
    print()
    print(y.decode('utf-8'))

os astrónomos acreditam que cada estrela da galáxia tem um planeta , e especulam que até um quinto deles tem um planeta do tipo da terra que poderá ter vida , mas ainda não vimos nenhum deles .

astronomers now believe that every star in the galaxy has a planet , and they speculate that up to one fifth of them have an earth-like planet that might be able to harbor life , but we have n't seen any of them .
